In [1]:
!pip install imdbpy

     |████████████████████████████████| 298 kB 4.1 MB/s 


In [2]:
from imdb import IMDb
from random import randint

In [3]:
# criando instância da classe IMDb
ia = IMDb()

In [4]:
def adicionarIndicacaoPerfil(indicacao, perfil_usuario):
  with open(perfil_usuario,'a') as arquivo: 
    arquivo.write(str(indicacao) + "\n")

In [5]:
def atualizarArquivo(arquivo_preenchido, perfil_usuario):
  with open(perfil_usuario, 'w+') as arquivo_atualizado:
    for item in arquivo_preenchido:
      arquivo_atualizado.write(str(item))

In [6]:
def leituraArquivo(perfil_usuario):
  with open(perfil_usuario, "r+") as arquivo:
      return arquivo.readlines()

In [7]:
  def removerRepeticoes(indicacao):
    arquivo_preenchido = leituraArquivo(arquivo_perfil)
    for linha in arquivo_preenchido:
      if indicacao['id_titulo'] in linha:
        arquivo_preenchido.remove(linha)

    atualizarArquivo(arquivo_preenchido, arquivo_perfil)	

In [8]:
def gerarObjetoIndicacao(titulo):
  id_titulo = titulo.getID()
  titulo_dados_completos = ia.get_movie(id_titulo)
  objeto_indicacao = {
    "id_titulo" : id_titulo if id_titulo != '' else '',
    "titulo_indicado": titulo_dados_completos['title'],
  }
  return objeto_indicacao

In [9]:
def retornarListaIndicacoesFiltroPorNomes(primeira_lista, segunda_lista, papel_pessoa):

  limite_indicacoes = 5
  lista_indicacao = []

  for titulo in primeira_lista['filmography'][papel_pessoa]:
    if titulo in segunda_lista:
      if limite_indicacoes > 0:
        objeto_indicacao = gerarObjetoIndicacao(titulo)
        lista_indicacao.append(objeto_indicacao)
        limite_indicacoes -= 1

  return lista_indicacao

In [10]:
def retornarListaIndicacoesFiltroAvancado(lista, papel_pessoa):

  limite_indicacoes = 5
  lista_indicacao = []

  for titulo in lista['filmography'][papel_pessoa]:
    if 'year' in titulo.keys() and titulo.data['year'] >= ano_inicial and titulo.data['year'] <= ano_final:
      id_titulo = titulo.getID()
      titulo_dados_completos = ia.get_movie(id_titulo)
      if titulo_genero in titulo_dados_completos['genres']:
        if 'rating' in titulo_dados_completos.keys() and titulo_dados_completos['rating'] == titulo_pontuacao:
          if titulo_premiacao == True:
            premiacao = ia.get_movie_awards(titulo.getID())
            if (premiacao['data'] != {}):
              objeto_indicacao = gerarObjetoIndicacao(titulo)
              lista_indicacao.append(objeto_indicacao)
            else:
              objeto_indicacao = gerarObjetoIndicacao(titulo)
              lista_indicacao.append(objeto_indicacao)
  return lista_indicacao

In [11]:
def gerarListaFilmografiaPessoa(pessoa):

  if pessoa:
    dados_pessoa = ia.search_person(pessoa) 
    lista = ia.get_person(dados_pessoa[0].personID)
  
  return lista

In [12]:

#@title **Crie seu perfil**
#@markdown Preencha o nome dos artistas para filtramos os títulos :

perfil = "'Luiz Henrique'" #@param {type: "string"}
arquivo_perfil = perfil + '.txt'
arquivo = open(arquivo_perfil, "w+")

In [13]:
def solicitarOpniaoPorIndicacao(lista):

  for iteracao, indicacao in enumerate(lista):
    if 'titulo_indicado' in indicacao.keys() : 
      print("Indicação " + str(iteracao + 1) + ": ", indicacao['titulo_indicado'])
      curtiu = int(input("Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) "))
      print('\n')
      if (curtiu == 0 or curtiu == 1):
        indicacao['curtiu'] = str(curtiu)
        removerRepeticoes(indicacao)  
        adicionarIndicacaoPerfil(indicacao, arquivo_perfil)   
      else:
        print("Erro, valor informado incorreto. Tente novamente!")
        break;

In [14]:
def retornarGeneroTituloPerfil(lista_generos):
  for genero in lista_generos:
    return genero

In [15]:
def retornarElencoTituloPerfil(titulo):
  for artista in titulo['cast']:
    return artista['name']

In [16]:
def retornarDiretorTituloPerfil(titulo):
  
  if 'directors' in titulo.keys():
    for director in titulo['directors']:
      return director['name']

In [17]:
def gerarIndicacoesArtistaGenero(artista_lista, papel_pessoa):

  limite_indicacoes = 1
  objeto_indicacao = {}

  for titulo in artista_lista['filmography'][papel_pessoa]:
    if titulo['title'] not in lista_titulo_perfil and (titulo['kind'] == 'movie' or titulo['kind'] == 'tv series'):
      if limite_indicacoes > 0:
        id_titulo = titulo.getID()
        titulo_dados_completos = ia.get_movie(id_titulo)
        if titulo_dados_completos['genres']:
          genero = retornarGeneroTituloPerfil(titulo_dados_completos['genres'])      
          if genero in set_generos:
            objeto_indicacao = gerarObjetoIndicacao(titulo)
            limite_indicacoes -= 1
  
  return objeto_indicacao


In [18]:
def gerarIndicacoesDiretorDecada(diretor_lista, papel_pessoa):

  limite_indicacoes = 1
  objeto_indicacao = {}

  for titulo in diretor_lista['filmography'][papel_pessoa]:
    if titulo['title'] not in lista_titulo_perfil and (titulo['kind'] == 'movie' or titulo['kind'] == 'tv series'):
      if limite_indicacoes > 0:
        if 'year' in titulo.keys():
          decada_titulo = titulo.data['year'] - (titulo['year'] % 10)
          if decada_titulo in set_decada:
            id_titulo = titulo.getID()
            titulo_dados_completos = ia.get_movie(id_titulo)
            diretor = retornarDiretorTituloPerfil(titulo_dados_completos)
            if diretor in set_diretor:
              objeto_indicacao = gerarObjetoIndicacao(titulo)
              limite_indicacoes -= 1
  
  return objeto_indicacao

In [20]:
#@title **Filtro de Pesquisa por artistas**
#@markdown Preencha o nome dos artistas para filtramos os títulos :

primeiro_artista = 'Robert Downey Jr' #@param {type: "string"}
segundo_artista = 'Chris Evans' #@param {type: "string"}

# pegar lista de filmografia 
if primeiro_artista and segundo_artista:
  lista_primeiro_artista = gerarListaFilmografiaPessoa(primeiro_artista)
  lista_segundo_artista = gerarListaFilmografiaPessoa(segundo_artista)

  papel_pessoa = "actress" if 'actress' in lista_primeiro_artista['filmography'].keys() else "actor"

  lista = retornarListaIndicacoesFiltroPorNomes(lista_primeiro_artista, lista_segundo_artista, papel_pessoa)
  if lista:
    solicitarOpniaoPorIndicacao(lista)
  else:
    print("Título(s) não encontrado(s)")

else:
  print("Erro! Campo(s) vazio(s)")


Indicação 1:  Avengers: Endgame
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1


Indicação 2:  Avengers: Infinity War
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1


Indicação 3:  Spider-Man: Homecoming
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1


Indicação 4:  Captain America: Civil War
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 0


Indicação 5:  Avengers: Age of Ultron
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1




In [ ]:
#@title **Filtro de Pesquisa por diretor e artista**
#@markdown Preencha o nome do diretor/diretora e artista para filtramos os títulos :

diretor = 'Quentin Tarantino' #@param {type: "string"}
artista = 'Samuel L Jackson' #@param {type: "string"}

# pegar lista de filmografia 
if diretor and artista:
  lista_diretor = gerarListaFilmografiaPessoa(diretor)
  lista_artista = gerarListaFilmografiaPessoa(artista)

  if 'director' in lista_diretor['filmography'].keys():
    papel_pessoa = "director"
    lista = retornarListaIndicacoesFiltroPorNomes(lista_diretor, lista_artista, papel_pessoa)

    if lista:
      solicitarOpniaoPorIndicacao(lista)
    else:
      print("Título(s) não encontrado(s)")
  else:
    print(diretor + " não possuí títulos como diretor/diretora. Tente novamente!")
    
else:
  print("Erro! Campo(s) vazio(s)")


Indicação 1:  The Hateful Eight


In [ ]:
#@title **Filtro de Pesquisa Avançada**

#@markdown Preencha os campos caso queira realizar pesquisa com filtro avançado

#@markdown 1. Nome do ator ou atriz:
artista_nome = 'Gal Gadot' #@param {type: "string"}

#@markdown 2. Lançamento (anos) 

#@markdown Informe o período que deseja procurar: 

#@markdown Entre
ano_inicial = 2010  #@param {type:"integer"}

#@markdown e
ano_final = 2021 #@param {type:"integer"}

#@markdown 3. Gênero: 
titulo_genero = "Action" #@param ["Action", "Adventure", "Animation", "Biography", "Comedy", "Crime", "Documentary", "Drama", "Family", "Fantasy", "Film Noir", "History", "Horror", "Music", "Musical", "Mystery", "Romance", "Sci-Fi", "Short Film", "Sport", "Superhero", "Thriller", "War", "Western"]

#@markdown 4. Pontuação: 

titulo_pontuacao = 8.8 #@param {type:"slider", min:0, max:10, step:0.1}

#@markdown 5. Premiação: 

#@markdown Informe se deseja procurar por um título que foi ou indicado e/ou ganhou prêmios:
titulo_premiacao = True #@param {type:"boolean"}

# pegar lista de filmografia 

if artista_nome and ano_inicial and ano_final:

  artista_lista = gerarListaFilmografiaPessoa(artista_nome)
  papel_pessoa = "actress" if 'actress' in artista_lista['filmography'].keys() else "actor"
  lista = retornarListaIndicacoesFiltroAvancado(artista_lista, papel_pessoa)
  
  if lista:
    solicitarOpniaoPorIndicacao(lista)
  else:
    print("Título(s) não encontrado(s)")
    
else:
  print("Erro! Campo(s) vazio(s)")

In [ ]:
#@title **(1) O que assistir?**

#@markdown **Artistas e gêneros**

#@markdown Filtro 1 : Títulos com mesmos artistas e gêneros dos títulos curtidos em seu perfil

lista_indicacao = leituraArquivo(arquivo_perfil)

lista_titulo_perfil = []
set_generos = set()
set_elenco = set()
lista_novas_indicacoes = []

for indicacao in lista_indicacao:
  linha = indicacao.split()
  id = linha[1].replace(',','').replace("'",'')
  titulo = ia.get_movie(id)

  lista_titulo_perfil.append(titulo['title'])

  if "'curtiu': '1'" in indicacao:

    set_generos.add(retornarGeneroTituloPerfil(titulo['genre']))
    set_elenco.add(retornarElencoTituloPerfil(titulo))

for contador in range(0,4):
  artista_nome = (list(set_elenco)[randint(0, len(set_elenco) - 1)])
  artista_lista = gerarListaFilmografiaPessoa(artista_nome)
  papel_pessoa = "actress" if 'actress' in artista_lista['filmography'].keys() else "actor"
  lista_novas_indicacoes.append(gerarIndicacoesArtistaGenero(artista_lista, papel_pessoa))


if lista_novas_indicacoes:
  solicitarOpniaoPorIndicacao(lista_novas_indicacoes)
else:
  print("Título(s) não encontrado(s)")

In [ ]:
#@title **(2) O que assistir?**

#@markdown **Diretores e décadas**

#@markdown Filtro 1 : Títulos com mesmos diretores e mesmas décadas dos títulos curtidos em seu perfil


lista_indicacao = leituraArquivo(arquivo_perfil)

diretor_lista = []
lista_titulo_perfil = []
set_decada = set()
set_diretor = set()
lista_novas_indicacoes = []

for indicacao in lista_indicacao:
  linha = indicacao.split()
  id = linha[1].replace(',','').replace("'",'')
  titulo = ia.get_movie(id)

  lista_titulo_perfil.append(titulo['title'])

  if "'curtiu': '1'" in indicacao and 'year' in titulo.keys():

    decada = titulo['year'] - (titulo['year'] % 10)
    set_decada.add(decada)
    set_diretor.add(retornarDiretorTituloPerfil(titulo))

for contador in range(0,4):
  diretor_nome = (list(set_diretor)[randint(0, len(set_diretor) - 1)])
  diretor_lista = gerarListaFilmografiaPessoa(diretor_nome)
  if 'director' in diretor_lista['filmography'].keys():
    papel_pessoa = "director"  
    lista_novas_indicacoes.append(gerarIndicacoesDiretorDecada(diretor_lista, papel_pessoa))

if lista_novas_indicacoes:
  solicitarOpniaoPorIndicacao(lista_novas_indicacoes)
else:
  print("Título(s) não encontrado(s)")

Indicação 1:  The Lord of the Rings
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1


Indicação 2:  Otherhalf
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 0


Indicação 3:  Battle at Big Rock
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 0


Indicação 4:  Fantastic Beasts: The Secrets of Dumbledore
Informe se curtiu a indicação colocando 1 (para sim) ou 0: (para não) 1


